# Basic data cleaning and tokenization

## Cleaning

Some simple, regex-based cleaning is performed on train and dev datasets, e.g. to remove HTML tags from Wikipedia articles, non-verbal cues from subtitles, or even to correct I’s that were incorrectly recognized as l’s in OCR’ed uppercase text.

In [1]:
from pathlib import Path
from mrclean import *

In [2]:
DATA_ROOT = Path("./")
SEQ_LENGTH = 128 # this is a legacy parameter, it does not affect cleaning
DATA_SPLITS = ['babylm_10M', 'babylm_dev']

CLEANUP_FUNCTIONS = {
    'aochildes': cleanup_aochildes,
    'bnc_spoken': cleanup_bnc_spoken,
    'cbt': cleanup_cbt,
    'children_stories': cleanup_children_stories,
    'gutenberg': cleanup_gutenberg,
    'open_subtitles': cleanup_open_subtitles,
    'qed': cleanup_qed,
    'simple_wikipedia': cleanup_simple_wikipedia,
    'switchboard': cleanup_switchboard,
    'wikipedia': cleanup_wikipedia,
}


In [3]:
for split in DATA_SPLITS:
    INPUT_DIR = DATA_ROOT / 'data' / split
    OUTPUT_DIR = DATA_ROOT / 'data' / f'{split}_clean'
    
    OUTPUT_DIR.mkdir(exist_ok=True)

    train_files = [f for f in INPUT_DIR.iterdir() if f.is_file() and f.suffix in ['.train', '.dev']]
    
    for file in train_files:
        text = file.read_text()
        cleaned_text = CLEANUP_FUNCTIONS[file.stem](text, SEQ_LENGTH)
        (OUTPUT_DIR / file.name).write_text(cleaned_text)
        print(f"🧹 Cleaned '{file.name}' (size {len(text)} -> {len(cleaned_text)}) in {split}")


🧹 Cleaned 'switchboard.train' (size 601050 -> 601050) in babylm_10M
🧹 Cleaned 'children_stories.train' (size 1797174 -> 1797174) in babylm_10M
🧹 Cleaned 'wikipedia.train' (size 6065862 -> 6008082) in babylm_10M
🧹 Cleaned 'gutenberg.train' (size 5745126 -> 5745126) in babylm_10M
🧹 Cleaned 'cbt.train' (size 2627694 -> 2554890) in babylm_10M
🧹 Cleaned 'open_subtitles.train' (size 16433872 -> 16431168) in babylm_10M
🧹 Cleaned 'bnc_spoken.train' (size 4493818 -> 4463546) in babylm_10M
🧹 Cleaned 'qed.train' (size 5781992 -> 5682028) in babylm_10M
🧹 Cleaned 'aochildes.train' (size 1900547 -> 1820547) in babylm_10M
🧹 Cleaned 'simple_wikipedia.train' (size 9087222 -> 9064456) in babylm_10M
🧹 Cleaned 'bnc_spoken.dev' (size 4451820 -> 4429688) in babylm_dev
🧹 Cleaned 'gutenberg.dev' (size 4893965 -> 4893965) in babylm_dev
🧹 Cleaned 'qed.dev' (size 5390732 -> 5295754) in babylm_dev
🧹 Cleaned 'switchboard.dev' (size 670013 -> 670013) in babylm_dev
🧹 Cleaned 'open_subtitles.dev' (size 15724635 -> 15

## Training a tokenizer

In [4]:
from pathlib import Path
from tokenizers import (Tokenizer, decoders, models, pre_tokenizers,
                        processors, trainers)
from tokenizers.normalizers import NFKC

In [7]:
# We train the tokenizer on the train data only
data_dir = Path("./data/babylm_10M_clean/")

paths = [str(f) for f in data_dir.glob("*") if f.is_file() and not f.name.endswith(".DS_Store") and f.suffix in [".train"]]

# paths
print(len(paths))
assert len(paths) > 0, 'No data files found'

10


In [8]:
tokenizer = Tokenizer(models.BPE())

tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=True)
tokenizer.decoder = decoders.ByteLevel()
tokenizer.post_processor = processors.ByteLevel(trim_offsets=True)
tokenizer.normalizer = NFKC()

In [9]:
trainer = trainers.BpeTrainer(vocab_size=16000, min_frequency=2, special_tokens=["<pad>", "<s>", "</s>"])
tokenizer.train(paths, trainer)

In [14]:
tokenizer_path =  DATA_ROOT / "models/gpt-clean-16000.json"
tokenizer.save(str(tokenizer_path), pretty=True)

## Testing the tokenizer

In [18]:

tokenizer = Tokenizer.from_file(str(tokenizer_path))


# text = 'Shiro Okada (岡田志郎, "Okada Shirō", June 9, 1949; Hirakata, Osaka {age 71} - ) is a Japanese guitarist who participate in the Group Sound band, the Ox. His nickname was Shiro (シロー) and his real name is Shiro Okamoto (岡田史郎).'
text = "The quick brown fox jumps over the lazy dog."

encoded = tokenizer.encode(text)
print(f"Encoded String: {encoded.tokens}")

print(f"Encoded IDs: {encoded.ids}")

decoded = tokenizer.decode(encoded.ids)
print(f"Decoded String: {decoded}")


Encoded String: ['ĠThe', 'Ġquick', 'Ġbrown', 'Ġfox', 'Ġj', 'umps', 'Ġover', 'Ġthe', 'Ġlazy', 'Ġdog', '.']
Encoded IDs: [295, 1993, 4629, 9277, 366, 8876, 574, 210, 12889, 2415, 16]
Decoded String:  The quick brown fox jumps over the lazy dog.
